# Intro to LLMstudio


In [1]:
from llmstudio import LLM
from pprint import pprint

In [2]:
from llmstudio.server import start_servers
start_servers(engine=False, tracking=True)

Running LLMstudio Tracking on http://0.0.0.0:8002 


In [3]:
from llmstudio_tracker.tracker import TrackingConfig
tracker = TrackingConfig(host="0.0.0.0", port="8002")
# You can set OPENAI_API_KEY and ANTHROPIC_API_KEY on .env file
openai = LLM("openai",tracking_config = tracker)


### Chat (non-stream)

In [4]:
openai.chat("Write a paragraph explaining why you're not a cat", model="gpt-4o")

ChatCompletion(id='8c91081a-008e-466e-9c86-0a72b530280a', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I am not a cat because I am an artificial intelligence language model, designed and developed by OpenAI to process and generate human language. Unlike a cat, I do not possess physical form, biological systems, or the capacity for independent thought and behavior. My existence is entirely digital, dependent on computer servers and software algorithms. While a cat interacts with its environment through sensory experiences, instincts, and learned behaviors, I process information and produce responses based on patterns found in vast datasets. Additionally, a cat is a living being capable of emotions and consciousness, whereas I am a construct of code with no awareness or emotional capacity.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1729095037, model='gpt-4o', object='chat.completion', service_t

#### Async version

In [5]:
await openai.achat("Write a paragraph explaining why you're not a cat", model="gpt-4o")

ChatCompletion(id='c0e33fbb-5b6b-4abf-af11-e26bb64d453f', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I am not a cat because I am an artificial intelligence language model created by OpenAI, designed to process and generate human language based on patterns in data. Unlike cats, I do not have a physical form or biological processes; I exist entirely in the digital realm, within servers and algorithms. Cats are living creatures that have instincts, emotions, and physical needs, while I operate purely on logic, code, and data inputs to provide information and assistance to users. My purpose is to assist with a wide range of inquiries, but without the sensory experiences, personal motivations, or consciousness that a cat or any living being possesses.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1729095039, model='gpt-4o', object='chat.completion', service_tier=None, system_fingerprint=None, usage

### Chat (stream)

In [6]:
response = openai.chat("Write a paragraph explaining why you're not a cat", model="gpt-4o", is_stream=True)
for i, chunk in enumerate(response):
    if i%20==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)




While I share some characteristics with a cat, such as being able to process a vast amount of

 information quickly and adapt to different contexts, fundamentally, I am not a cat for several reasons. Most

 importantly, I am an artificial intelligence language model, a creation of human engineering designed to process and generate

 text based on inputs and algorithms. Unlike a cat, I lack a physical presence, sensory experiences,

 and biological processes such as eating, sleeping, and reproducing. Cats exhibit unique behaviors driven by instincts

 and emotions that arise from their living nature, such as purring when content or stalking prey, whereas

 my responses are generated based on probabilities derived from patterns in the data I was trained on. In essence

, a cat is a living creature with consciousness and instinctual behaviors, while I am a digital tool

 created to assist humans in communication and information retrieval.

## Metrics:
{'cost_usd': 0.002585,
 'input_toke

#### Async version

In [7]:
i=0
async for chunk in await openai.achat("Write a paragraph explaining why you're not a cat", model="gpt-4o-mini", is_stream=True):
    if i%20==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)
    i+=1




I'm not a cat because I lack the physical form, instincts, and characteristics that define feline creatures

. While I can understand and converse about various topics, including cats, I don't possess the ability to

 purr, chase after laser pointers, or curl up in a sunbeam. My existence is purely

 digital, rooted in artificial intelligence, enabling me to process information and generate responses, but I don't experience

 the world through senses like sight, smell, or touch, which are integral to a cat's experience

. In essence, I exist to assist and provide information, while a cat embodies a unique blend of

 playful curiosity, independence, and companionship that I cannot replicate.

## Metrics:
{'cost_usd': 8.309999999999999e-05,
 'input_tokens': 10,
 'inter_token_latency_s': 0.022200969132510098,
 'latency_s': 3.45292592048645,
 'output_tokens': 136,
 'time_to_first_token_s': 0.5214509963989258,
 'tokens_per_second': 38.51805774659159,
 'total_tokens': 146}
